In [1]:
import sqlite3
import requests
from xml.etree import ElementTree
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
server = "https://www.wms.nrw.de/wms/elwas-ow?REQUEST=GetFeatureInfo&VERSION=1.1.1&SRS=EPSG:25832&BBOX=-519722.2005900545,5455569.10294,1329442.3876849292,5942320.099060002&WIDTH=1512&HEIGHT=398&LAYERS=badegewaesser_eu&QUERY_LAYERS=badegewaesser_eu&FORMAT=image/png&INFO_FORMAT=application/vnd.ogc.gml&FEATURE_COUNT=500&RADIUS=500&X=787&Y=199"

In [3]:
response = requests.get(server)
data = ElementTree.fromstring(response.content)


In [4]:
locationData = []
namespaces = {
    'gml': 'http://www.opengis.net/gml'
}


for location in data[0].findall('badegewaesser_eu_feature'):
    id = location.find('messstelle_id').text
    name = location.find('badegewaesser_bezeichnung').text
    alternate_name = ' '.join([name, location.find('messstelle_bezeichnung').text])
    bbox = location.find('./gml:boundedBy/gml:Box/gml:coordinates', namespaces=namespaces).text
    coords = bbox.split(" ")[0].split(",")
    locationData.append([
        id, name, alternate_name, 
        Point(coords[0], coords[1])
    ])


In [5]:
df = gpd.GeoDataFrame(locationData, columns=["id", "name", "alternate_name", "bbox"])\
.set_index('id')
df

,name,alternate_name,bbox
id,,,
1,Unterbacher See,Unterbacher See Nord,POINT (352644 5673856)
2,Unterbacher See,Unterbacher See Süd,POINT (352317 5673635)
3,Kruppsee,Kruppsee Freibad,POINT (340153 5695735)
4,Wolfssee,Wolfssee Freibad,POINT (346829 5694789)
5,Großenbaumer See,Großenbaumer See Freizeitanlage,POINT (344826 5692376)
...,...,...,...
76,Listertalsperre,Listertalsperre Camping Windebruch,POINT (417164 5659189)
77,Listertalsperre,Listertalsperre Rengerberg,POINT (416092 5659142)
84,Möhnesee,Möhnesee Strandbad Delecke,POINT (436242 5704792)


In [6]:
df = df.set_geometry('bbox')
df = df.set_crs('25832')

In [7]:
df.crs

<Projected CRS: EPSG:25832>
Name: ETRS89 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 6°E and 12°E: Austria; Belgium; Denmark - onshore and offshore; Germany - onshore and offshore; Norway including - onshore and offshore; Spain - offshore.
- bounds: (6.0, 38.76, 12.01, 84.33)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
df

,name,alternate_name,bbox
id,,,
1,Unterbacher See,Unterbacher See Nord,POINT (352644.000 5673856.000)
2,Unterbacher See,Unterbacher See Süd,POINT (352317.000 5673635.000)
3,Kruppsee,Kruppsee Freibad,POINT (340153.000 5695735.000)
4,Wolfssee,Wolfssee Freibad,POINT (346829.000 5694789.000)
5,Großenbaumer See,Großenbaumer See Freizeitanlage,POINT (344826.000 5692376.000)
...,...,...,...
76,Listertalsperre,Listertalsperre Camping Windebruch,POINT (417164.000 5659189.000)
77,Listertalsperre,Listertalsperre Rengerberg,POINT (416092.000 5659142.000)
84,Möhnesee,Möhnesee Strandbad Delecke,POINT (436242.000 5704792.000)


In [9]:
df = df.to_crs('4326')
df

,name,alternate_name,bbox
id,,,
1,Unterbacher See,Unterbacher See Nord,POINT (6.89095 51.19709)
2,Unterbacher See,Unterbacher See Süd,POINT (6.88637 51.19502)
3,Kruppsee,Kruppsee Freibad,POINT (6.70253 51.39031)
4,Wolfssee,Wolfssee Freibad,POINT (6.79881 51.38365)
5,Großenbaumer See,Großenbaumer See Freizeitanlage,POINT (6.77110 51.36142)
...,...,...,...
76,Listertalsperre,Listertalsperre Camping Windebruch,POINT (7.81748 51.07823)
77,Listertalsperre,Listertalsperre Rengerberg,POINT (7.80219 51.07765)
84,Möhnesee,Möhnesee Strandbad Delecke,POINT (8.08164 51.49068)


In [10]:
locations = df.copy()
locations['long'] = df.bbox.apply(lambda p: p.x)
locations['lat'] = df.bbox.apply(lambda p: p.y)
locations = locations.drop(columns=['bbox'])
locations

,name,alternate_name,long,lat
id,,,,
1,Unterbacher See,Unterbacher See Nord,6.890953,51.197087
2,Unterbacher See,Unterbacher See Süd,6.886367,51.195016
3,Kruppsee,Kruppsee Freibad,6.702527,51.390308
4,Wolfssee,Wolfssee Freibad,6.798811,51.383650
5,Großenbaumer See,Großenbaumer See Freizeitanlage,6.771103,51.361424
...,...,...,...,...
76,Listertalsperre,Listertalsperre Camping Windebruch,7.817482,51.078225
77,Listertalsperre,Listertalsperre Rengerberg,7.802193,51.077647
84,Möhnesee,Möhnesee Strandbad Delecke,8.081644,51.490675


In [11]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")

112